In [1]:
import torch
import pandas as pd
from torch_geometric.data import Data
import torch
from torch_geometric.data import Data
import pandas as pd
import pickle
import numpy as np

from torch_geometric.nn import SAGEConv
import torch
import torch_geometric
import torch.nn.functional as F
torch_geometric.set_debug(True)
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pandas as pd
from utils import bert_encode
import json
from tqdm import tqdm  # make sure to install with: pip install tqdm

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import pandas as pd
path_disease_symptom_csv = "data/disease_knowledge_graph/Final_Augmented_dataset_Diseases_and_Symptoms.csv"
disease_symptom_csv =  pd.read_csv(path_disease_symptom_csv)

# get first few rows
disease_symptom_csv.head()

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
import torch
import pandas as pd
path_patient_doctor_csv = "data/disease_knowledge_graph/patient-doctor.csv"
patient_doctor_csv =  pd.read_csv(path_patient_doctor_csv)

# get first few rows
patient_doctor_csv.head()

,PD_ID,Description,Patient,Doctor,problem_description
0,0,How can one treat intense pain in the stomach ...,"Hi,I m XXXX,I am ulcer patient ,I did my endos...","<start> Hello,I don't think your ulcer is comi...",How can one treat intense pain in the stomach ...
1,1,What causes abdominal pain similar to menstrua...,I took the shot and started duphaston pills fo...,"<start> Hello,Duphaston is a hormonal preparat...",What causes abdominal pain similar to menstrua...
2,2,Suggest remedies for recurrent abdominal pain ...,Hi my name is XXXX. I have been having stomach...,"<start> Hello,I read carefully your query and ...",Suggest remedies for recurrent abdominal pain ...
3,3,What causes sensation of weird movements in th...,"Hi, my name is XXXX I m a 19year old girl and ...","<start> Hi,There can be numerous clinical cond...",What causes sensation of weird movements in th...
4,4,Suggest remedies for recurrent pain in the abd...,Hi! My name is XXXX and I used to take an insa...,"<start> Hello,Some females get more gastritis ...",Suggest remedies for recurrent pain in the abd...


In [12]:
import torch
import pandas as pd
from torch_geometric.data import Data
import torch
from torch_geometric.data import Data
import pandas as pd
import pickle
import numpy as np

from torch_geometric.nn import SAGEConv
import torch
import torch_geometric
import torch.nn.functional as F
torch_geometric.set_debug(True)
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pandas as pd
from utils import bert_encode
import json
from tqdm import tqdm  # make sure to install with: pip install tqdm

import re
import json
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
import torch_geometric
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # pip install tqdm
from transformers import AutoTokenizer, AutoModel

from utils import bert_encode  # custom function

torch_geometric.set_debug(True)


class DiseaseKnowledgeGraph:
    def __init__(
        self,
        main_graph_path,
        disease_symptom_csv_path,
        patient_doctor_csv_path,
        disease_list_path,
        symptom_list_path,
        disease_aliases_path=None,
        graphs=None
    ):
        
        if graphs is not None:
            self.graphs = graphs
            return

        # === Load data ===
        self.main_graph_df = pd.read_csv(main_graph_path)
        self.disease_symptom_csv = pd.read_csv(disease_symptom_csv_path)
        self.patient_doctor_csv = pd.read_csv(patient_doctor_csv_path)

        # Load disease list & mapping
        self.disease_list = sorted(self.main_graph_df["diseases"].dropna().unique().tolist())
        print(f'Number of unique diseases: {len(self.disease_list)}')
        self.disease2id = {disease: i for i, disease in enumerate(self.disease_list)}

        # Load symptoms
        self.symptom_list = pd.read_csv(symptom_list_path)["symptoms"].tolist()
        self.alias_disease_list = pd.read_csv(disease_list_path)["0"].tolist()

        # Load disease aliases
        self.disease2aliases = self.load_disease_aliases(disease_aliases_path)
        if self.disease2aliases:
            self.alias2disease = {
                alias.lower(): disease
                for disease, aliases in self.disease2aliases.items()
                for alias in aliases
            }
        else:
            self.alias2disease = {}

        # === Create main graph ===
        self.main_graph, self.node2id, self.num_node_features = self.create_graph(self.main_graph_df)
        self.graphs = []
        
        # === Build patient-doctor graphs ===
        for disease in tqdm(self.disease_list, desc="Building disease-symptom graphs"):
            # Get all rows for this disease
            disease_rows = self.disease_symptom_csv[self.disease_symptom_csv["diseases"] == disease]

            # Sample up to 20 random examples (if fewer than 20, take all)
            sampled_rows = disease_rows.sample(n=min(4, len(disease_rows)), random_state=42)

            for _, row_data in sampled_rows.iterrows():
                symptoms = [
                    self.symptom_list[i]
                    for i, val in enumerate(row_data[1:].tolist())  # assuming first col is "diseases"
                    if val == 1
                ]

                data = self.encode_text_and_match_diseases("person")
                data["exact_matches_symptom"].extend(symptoms)
                self.graphs.append(self.update_graph_data(data, disease))


        for _, row_data in tqdm(
            self.patient_doctor_csv.sample(n=1500, random_state=42).iterrows(),
            total=1500,
            desc="Building patient-doctor graphs"
        ):
            text_patient = row_data["problem_description"]
            data_patient = self.encode_text_and_match_diseases(text_patient)

            text_doctor = row_data["Doctor"]
            data_doctor = self.encode_text_and_match_diseases(text_doctor)

            disease = data_doctor["exact_matches_disease"]
            if len(disease) == 0:
                disease_in_list = data_doctor["semantic_matches_disease"][0:1]
                disease = [disease_in_list[0][0]]
            self.graphs.append(self.update_graph_data(data_patient, disease))

    def load_disease_aliases(self, disease_aliases_path):
        if disease_aliases_path is not None:
            with open(disease_aliases_path, "r") as f:
                return json.load(f)
        return {}

    def encode_text_and_match_diseases(self, text, top_k=5):
        encoding_dict = {}
        text_lower = text.lower()
        lists = [("disease", self.alias_disease_list), ("symptom", self.symptom_list)]
        text_vec = bert_encode(text)
        encoding_dict["text_vector"] = text_vec[0].tolist()

        for list_type, lst in lists:
            lst_lower = [item.lower() for item in lst]

            # Exact matches
            exact_matches = [
                entity for entity in lst_lower
                if re.search(rf"\b{re.escape(entity)}\b", text_lower)
            ]

            # Semantic matches
            entity_vecs = np.vstack([bert_encode(entity) for entity in lst_lower])
            similarities = cosine_similarity(text_vec, entity_vecs)[0]
            top_indices = similarities.argsort()[-top_k:][::-1]
            semantic_matches = [(lst_lower[i], similarities[i]) for i in top_indices]

            encoding_dict[f"exact_matches_{list_type}"] = exact_matches
            encoding_dict[f"semantic_matches_{list_type}"] = semantic_matches

        return encoding_dict

    def create_graph(self, df):
        person_node = "person"
        disease_nodes = df["diseases"].unique().tolist()
        symptom_nodes = df.columns[1:].tolist()

        all_nodes = [person_node] + disease_nodes + symptom_nodes
        node2id = {name: i for i, name in enumerate(all_nodes)}

        mask = np.zeros((len(all_nodes),), dtype=bool)
        for node in disease_nodes:
            mask[node2id[node]] = 1
        
        edges, edge_types = [], []

        # person → disease (type 0)
        for disease in disease_nodes:
            edges.append([node2id[person_node], node2id[disease]])
            edge_types.append(0)

        # disease → symptom (type 1)
        for _, row in df.iterrows():
            disease = row["diseases"]
            for symptom in symptom_nodes:
                if row[symptom] == 1:
                    edges.append([node2id[disease], node2id[symptom]])
                    edge_types.append(1)

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_types, dtype=torch.long)

        # === Node features ===
        node_features = []
        for node in all_nodes:
            vec = bert_encode(node)
            if isinstance(vec, (np.ndarray, torch.Tensor)):
                vec = vec.flatten().tolist()
            else:
                vec = list(vec)

            if node == person_node:
                node_features.append([0, 1] + vec)
            elif node in disease_nodes:
                node_features.append([1, 0] + vec)
            else:
                node_features.append([2, 0] + vec)

        x = torch.tensor(node_features, dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        
        data.disease_mask = torch.tensor(mask, dtype=torch.bool)
        return data, node2id, x.shape[1]

    def update_graph_data(self, new_data, list_y):
        gnn_graph = self.main_graph.clone()
        disease_list = (
            new_data.get("exact_matches_disease", [])
            + [d for d, _ in new_data.get("semantic_matches_disease", [])]
        )
        symptoms_list = (
            new_data.get("exact_matches_symptom", [])
            + [s for s, _ in new_data.get("semantic_matches_symptom", [])]
        )
        vector = new_data["text_vector"]

        x = gnn_graph.x.clone().numpy()
        x[0] = [0, 1] + vector

        for disease in disease_list:
            if disease in self.alias2disease:
                x[self.node2id[self.alias2disease[disease]], 1] = 1
        
        edges = gnn_graph.edge_index.numpy()
        for symptom in symptoms_list:
            if symptom in self.node2id:
                x[self.node2id[symptom], 1] = 1
                symptom_id = self.node2id[symptom]
                edges = np.hstack([edges, np.array([[0], [symptom_id]])])
            

        gnn_graph.x = torch.tensor(x, dtype=torch.float)
        gnn_graph.edge_index = torch.tensor(edges, dtype=torch.long)
        
        y = np.zeros(len(self.disease_list), dtype=np.float32)
        for disease in list_y:
            if disease in self.disease2id:
                y[self.disease2id[disease]] = 1
        gnn_graph.y = torch.tensor(y, dtype=torch.float32)

        return gnn_graph

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]


In [4]:


main_graph_path = "data/disease_knowledge_graph/disease_csv_files/diseases_symptoms_merged.csv"
disease_symptom_csv_path = "data/disease_knowledge_graph/Final_Augmented_dataset_Diseases_and_Symptoms.csv"
patient_doctor_csv_path = "data/disease_knowledge_graph/patient-doctor.csv"
disease_list_path = "data/disease_knowledge_graph/disease_csv_files/unique_aliases.csv"
disease_aliases_path = "data/disease_knowledge_graph/disease_aliases.json"
symptom_list_path = "data/disease_knowledge_graph/disease_csv_files/unique_symptoms.csv"



In [ ]:
# dont run
disease_graphs = DiseaseKnowledgeGraph(main_graph_path, disease_symptom_csv_path, patient_doctor_csv_path, disease_list_path, symptom_list_path, disease_aliases_path)

In [5]:
# Load dataset from file
with open("disease_graphs.pkl", "rb") as f:
    loaded_graphs = pickle.load(f)

print(f"Total graphs loaded: {len(loaded_graphs)}")
print(f"the x of the first graph is {loaded_graphs[1]}")



Total graphs loaded: 4506
the x of the first graph is Data(x=[1151, 386], edge_index=[2, 6171], edge_attr=[6162], disease_mask=[1151], y=[773])


In [6]:
# Load dataset from file
with open("disease_graphs_qa.pkl", "rb") as f:
    loaded_graphs_qa = pickle.load(f)

print(f"Total graphs loaded: {len(loaded_graphs_qa)}")
print(f"the x of the first graph is {loaded_graphs_qa[1]}")


Total graphs loaded: 1500
the x of the first graph is Data(x=[1151, 386], edge_index=[2, 6168], edge_attr=[6162], disease_mask=[1151], y=[773])


In [13]:
all_graphs = loaded_graphs.graphs + loaded_graphs_qa.graphs
print(f"Total combined graphs: {len(all_graphs)}")

all_disease_graphs = DiseaseKnowledgeGraph(
        None,
        None,
        None, 
        disease_list_path,
        symptom_list_path,
        disease_aliases_path,
        graphs=all_graphs
    )  


Total combined graphs: 6006


In [14]:
# save to file
with open("all_disease_graphs.pkl", "wb") as f:
    pickle.dump(all_disease_graphs, f)

In [7]:
from gnn import DiseaseSymptomGraphGNN  # your GNN model

In [15]:

import json
import re
import pickle
from typing import Dict, List, Tuple, Optional
import random


import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv
from gnn import DiseaseSymptomGraphGNN  # your GNN model

from utils import bert_encode  # your custom encoder

torch_geometric.set_debug(True)

def set_seed(seed: Optional[int]):
    if seed is None:
        return
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def _infer_disease_mask_from_x(x: torch.Tensor) -> torch.Tensor:
    """
    Fallback: infer disease nodes from the first feature == 1.0 as per encoding [1, 0] + ...
    """
    # Robust check: avoid fp precision issues by a small tolerance
    return torch.isclose(x[:, 0], torch.tensor(1.0, device=x.device))

from sklearn.model_selection import train_test_split
import os

def train_disease_gnn(
    dataset_path: str,
    dict_paths: Dict[str, str],
    disease_symptom_df_path: str,
    hidden_channels: int = 128,
    out_channels: int = 1,
    num_layers: int = 2,
    dropout: float = 0.5,
    batch_size: int = 16,
    num_epochs: int = 10,
    lr: float = 1e-3,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    seed: int = 42,
    pos_loss_weight: float = 0.5,
    model_save_path: str = "best_model.pt", 
):
    set_seed(seed)

    # === Load dataset ===
    with open(dataset_path, "rb") as f:
        dataset = pickle.load(f)

    # First split: train_val vs test
    train_val_data, test_data = train_test_split(dataset, test_size=0.2, random_state=seed)

    # Second split: train vs validation
    train_data, val_data = train_test_split(train_val_data, test_size=0.2, random_state=seed)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # === Build model ===
    df = pd.read_csv(disease_symptom_df_path)
    model = DiseaseSymptomGraphGNN(
        df=df,
        dict_paths=dict_paths,
        hidden_channels=hidden_channels,
        out_channels=out_channels,
        num_layers=num_layers,
        dropout=dropout,
        seed=seed,
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    mse = nn.MSELoss()

    best_val_goal = 0.0
    best_model_path = model_save_path

    # === Training loop ===
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        # Random multipliers for this epoch (between 0.5 and 1.0)
        w_pos = random.uniform(0.5, 1.0)
        w_mse = random.uniform(0.5, 1.0)

        print(f"Epoch {epoch+1}: Loss Weights -> pos={w_pos:.3f}, mse={w_mse:.3f}")

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            logits = model.forward_pass(batch.x, batch.edge_index)
            disease_mask = getattr(batch, "disease_mask", _infer_disease_mask_from_x(batch.x))

            logits_disease = logits[disease_mask]
            targets = batch.y.float().unsqueeze(-1)

            # Loss components
            base_loss = criterion(logits_disease, targets)

            pos_mask = (targets == 1)
            if pos_mask.any():
                pos_logits = logits_disease[pos_mask]
                pos_targets = targets[pos_mask]
                pos_loss = criterion(pos_logits, pos_targets)
            else:
                pos_loss = torch.tensor(0.0, device=device)

            soft_max_logits = torch.softmax(logits_disease, dim=-1)
            mse_loss = mse(soft_max_logits, targets)

            # Weighted sum with randomized weights
            loss =  base_loss + w_pos * pos_loss + w_mse * mse_loss
            loss.backward()
            optimizer.step()

            total_loss += float(loss.item())

        avg_loss = total_loss / max(1, len(train_loader))

        # === Validation ===
        model.eval()
        correct, total = 0, 0
        correct_pos, total_pos = 0, 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                logits = model.forward_pass(batch.x, batch.edge_index)
                disease_mask = getattr(batch, "disease_mask", _infer_disease_mask_from_x(batch.x))

                logits_disease = logits[disease_mask]
                targets = batch.y.float().unsqueeze(-1)
                preds = (torch.sigmoid(logits_disease) > 0.5).long()

                correct += (preds.cpu() == targets.cpu().long()).sum().item()
                total += targets.numel()

                pos_mask = (targets == 1)
                if pos_mask.any():
                    correct_pos += (preds[pos_mask].cpu() == targets[pos_mask].cpu().long()).sum().item()
                    total_pos += pos_mask.sum().item()

        val_acc = correct / total if total > 0 else 0.0
        val_pos_acc = min(correct_pos / total_pos if total_pos > 0 else 0.0, 0.7)
        alpha = 0.7  # weight for val_acc (0.5 = equal, >0.5 = more emphasis on val_acc)
        beta = 1 - alpha

        val_goal = (1 + alpha) * (val_acc * val_pos_acc) / (alpha * val_acc + beta * val_pos_acc + 1e-8)



        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_loss:.4f} "
              f"- Val Acc: {val_acc:.4f} - Val Pos Acc: {val_pos_acc:.4f} - Val Goal: {val_goal:.4f}")

        # Save best model
        if val_goal > best_val_goal:
            best_val_goal = val_goal
            torch.save(model.state_dict(), best_model_path)
            print(f"✅ Saved new best model at epoch {epoch+1} with Val Goal {val_goal:.4f}")

        # Every 5 epochs, reset to best model if not best
        if (epoch + 1) % 5 == 0:
            if val_goal < best_val_goal:
                model.load_state_dict(torch.load(best_model_path))
                print(f"🔄 Reverted to best model (Val Goal {best_val_goal:.4f}) at epoch {epoch+1}")

    # === Load best model before final test ===
    model.load_state_dict(torch.load(model_save_path))
    model.eval()

    # === Final Test Evaluation ===
    correct, total, correct_pos, total_pos = 0, 0, 0, 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            logits = model.forward_pass(batch.x, batch.edge_index)
            disease_mask = getattr(batch, "disease_mask", _infer_disease_mask_from_x(batch.x))

            logits_disease = logits[disease_mask]
            targets = batch.y.float().unsqueeze(-1)
            preds = (torch.sigmoid(logits_disease) > 0.5).long()

            correct += (preds.cpu() == targets.cpu().long()).sum().item()
            total += targets.numel()

            pos_mask = (targets == 1)
            if pos_mask.any():
                correct_pos += (preds[pos_mask].cpu() == targets[pos_mask].cpu().long()).sum().item()
                total_pos += pos_mask.sum().item()

    accuracy = correct / total if total > 0 else 0.0
    pos_accuracy = correct_pos / total_pos if total_pos > 0 else 0.0

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Positive-case Accuracy: {pos_accuracy:.4f}")

    return model, accuracy, pos_accuracy


In [16]:
dataset_path = "all_disease_graphs.pkl"
disease_symptom_df_path = "data/disease_knowledge_graph/disease_csv_files/diseases_symptoms_merged.csv"

dict_paths = {"main_graph_path" : "data/disease_knowledge_graph/disease_csv_files/diseases_symptoms_merged.csv",
"disease_symptom_csv_path" :  "data/disease_knowledge_graph/Final_Augmented_dataset_Diseases_and_Symptoms.csv",
"patient_doctor_csv_path" : "data/disease_knowledge_graph/patient-doctor.csv",
"disease_list_path" : "data/disease_knowledge_graph/disease_csv_files/unique_aliases.csv",
"disease_aliases_path" : "data/disease_knowledge_graph/disease_aliases.json",
"symptom_list_path" : "data/disease_knowledge_graph/disease_csv_files/unique_symptoms.csv"}

model, acc, pos_acc = train_disease_gnn(dataset_path,
    dict_paths,
    disease_symptom_df_path,
    hidden_channels=128,
    out_channels=1,
    num_layers=2,
    dropout=0.6,
    batch_size=16,
    num_epochs=40,
    lr=1e-3,
    device="cuda" if torch.cuda.is_available() else "cpu",
    seed=42)

Number of unique diseases: 773
Number of disease nodes in model graph: 773
Epoch 1: Loss Weights -> pos=0.820, mse=0.513
Epoch 1/40 - Train Loss: 1.3011 - Val Acc: 0.7557 - Val Pos Acc: 0.7000 - Val Goal: 1.2169
✅ Saved new best model at epoch 1 with Val Goal 1.2169
Epoch 2: Loss Weights -> pos=0.638, mse=0.612
Epoch 2/40 - Train Loss: 1.1973 - Val Acc: 0.8971 - Val Pos Acc: 0.7000 - Val Goal: 1.2740
✅ Saved new best model at epoch 2 with Val Goal 1.2740
Epoch 3: Loss Weights -> pos=0.868, mse=0.838
Epoch 3/40 - Train Loss: 1.4708 - Val Acc: 0.8312 - Val Pos Acc: 0.7000 - Val Goal: 1.2492
Epoch 4: Loss Weights -> pos=0.946, mse=0.543
Epoch 4/40 - Train Loss: 1.1435 - Val Acc: 0.8125 - Val Pos Acc: 0.7000 - Val Goal: 1.2416
Epoch 5: Loss Weights -> pos=0.711, mse=0.515
Epoch 5/40 - Train Loss: 1.0070 - Val Acc: 0.9030 - Val Pos Acc: 0.7000 - Val Goal: 1.2760
✅ Saved new best model at epoch 5 with Val Goal 1.2760
Epoch 6: Loss Weights -> pos=0.609, mse=0.753
Epoch 6/40 - Train Loss: 1.17

In [ ]:

model.text_forward("Hello doctor,I have a friend, 22 years old, who has been struggling with anxiety lately. When he was a kid, he had ADHD attention deficit hyperactivity disorder and went through some therapies for it. Now, he is dealing with anxiety disorder, and it is really getting to him. I have heard about cognitive behavioral therapy as a way to manage anxiety, but I am not sure if it is suitable for him, given his history with ADHD. Could therapy help him with his anxiety, considering his past experiences with therapy? Also, could you tell me a bit more about what therapy involves and how effective it might be for someone in his situation?Thank you.", top_k=10)

['anxiety',
 'noninfectious gastroenteritis',
 'depression',
 'dissociative disorder',
 'open wound of the mouth',
 'pain after an operation',
 'flu',
 'factitious disorder',
 'persistent vomiting of unknown cause',
 'pregnancy']

In [25]:
dataset_path = "all_disease_graphs.pkl"
disease_symptom_df_path = "data/disease_knowledge_graph/disease_csv_files/diseases_symptoms_merged.csv"

dict_paths = {"main_graph_path" : "data/disease_knowledge_graph/disease_csv_files/diseases_symptoms_merged.csv",
"disease_symptom_csv_path" :  "data/disease_knowledge_graph/Final_Augmented_dataset_Diseases_and_Symptoms.csv",
"patient_doctor_csv_path" : "data/disease_knowledge_graph/patient-doctor.csv",
"disease_list_path" : "data/disease_knowledge_graph/disease_csv_files/unique_aliases.csv",
"disease_aliases_path" : "data/disease_knowledge_graph/disease_aliases.json",
"symptom_list_path" : "data/disease_knowledge_graph/disease_csv_files/unique_symptoms.csv"}

df = pd.read_csv(disease_symptom_df_path)
model_text_forward = DiseaseSymptomGraphGNN(
    df=df,
    dict_paths=dict_paths,
    hidden_channels=128,
    out_channels=1,
    num_layers=2,
    dropout=0.5,
    seed=42,
)

model_text_forward.load_state_dict(torch.load("best_model.pt", map_location=torch.device('cpu')))
model_text_forward.eval()
model_text_forward.text_forward("Hello doctor,I have a friend, 22 years old, who has been struggling with anxiety lately. When he was a kid, he had ADHD attention deficit hyperactivity disorder and went through some therapies for it. Now, he is dealing with anxiety disorder, and it is really getting to him. I have heard about cognitive behavioral therapy as a way to manage anxiety, but I am not sure if it is suitable for him, given his history with ADHD. Could therapy help him with his anxiety, considering his past experiences with therapy? Also, could you tell me a bit more about what therapy involves and how effective it might be for someone in his situation?Thank you.", top_k=10)

Number of unique diseases: 773
Number of disease nodes in model graph: 773


['anxiety',
 'noninfectious gastroenteritis',
 'depression',
 'dissociative disorder',
 'open wound of the mouth',
 'pain after an operation',
 'flu',
 'factitious disorder',
 'persistent vomiting of unknown cause',
 'pregnancy']